In [3]:
import prepare
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import nltk
from collections import Counter
stemmer = nltk.stem.PorterStemmer()
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import pickle
import re
import json
import zipfile
import logging
import os
import unicodedata
# import preprocess
import re
import json
import os
import sys
from collections import defaultdict
import nltk
from collections import Counter
from math import log, sqrt
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
################################################################
# ENVIRON
################################################################
# This is server-gpu
server1_homepath = "/home/ubuntu/workspace/codelab/"
server2_homepath = "/home/ubuntu/workspace/codelab/"
gpu_homepath = "/home/shawn/workspace/research/final_codelab/"
jun_homepath = "/home/junw/workspace/codelab/"

# choose from the server1, server2, gpu, jun.
SERVERNAME = 'gpu'
HOMEPATH = {'server1':server1_homepath, 'server2':server2_homepath, 'gpu':gpu_homepath, 'jun':jun_homepath}[SERVERNAME]

TASKNAME = 'preprocess'

## get the path of the wiki files
DATAPATH = HOMEPATH + "data/"
ORGINAL_DATAPATH = DATAPATH +"orignal/"
INTERMEDIATE_DATAPATH = DATAPATH + "intermediate/"
FINAL_DATAPATH =  DATAPATH + "final/"
################################################################
# ENVIRON
################################################################

In [4]:
train,dev,test = prepare.get_training_devset_test(ORGINAL_DATAPATH)
wiki = prepare.Wiki(DATAPATH, True ,INTERMEDIATE_DATAPATH )

100%|██████████| 109/109 [17:38<00:00, 10.23s/it]


In [10]:
dir(wiki)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'alltitles',
 'dertitles',
 'get_upper_title',
 'multi_docs',
 'multi_sents',
 'search',
 'search_tree',
 'single_doc',
 'single_sent',
 'title_tree',
 'wiki',
 'wiki_titles']

In [22]:
wiki.single_sent(['Magic_Johnson', 0])

"Magic Johnson , Earvin '' Magic '' Johnson Jr. -LRB- born August 14 , 1959 -RRB- is an American retired professional basketball player and current president of basketball operations of the Los Angeles Lakers of the National Basketball Association -LRB- NBA -RRB- ."

In [12]:
type(wiki)

prepare.Wiki

In [5]:
import re
import json
import os
import sys
from collections import defaultdict
import nltk
from collections import Counter
from math import log, sqrt
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import unicodedata
import logging
from nltk.stem import WordNetLemmatizer
import csv
from tqdm import tqdm
from nltk.corpus import stopwords

def preprocessed_claim_sentence(claim):
    claim = unicodedata.normalize('NFC', claim)
    claim = claim.replace(':','-COLON-')
    claim = claim.replace('-COLON-',' -COLON-')
    claim = claim.replace('(','-LRB-')
    claim = claim.replace(')','-RRB-')
    claim = claim.replace("_"," ").replace("-LRB-","-LRB- ").replace("-RRB-"," -RRB")
    claim = re.sub('–', '-', claim)
    claim = claim.replace("`","'")
    claim = claim.replace("  "," ")
    # replaced = re.sub('_-LRB-.*', '', title)
    return claim.strip()

class InvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.max_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.max_doc_len = max(doc_len, self.max_doc_len)
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                term_id = vocab[term]
                self.doc_ids[term_id].append(docid)
                self.doc_term_freqs[term_id].append(freq)
                self.doc_freqs[term_id] += 1

    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        return self.doc_ids[term_id]

    def freqs(self, term):
        term_id = self.vocab[term]
        return self.doc_term_freqs[term_id]

    def f_t(self, term):
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]
    
def processed(docs, lower = False, stem = False):
# processed_docs stores the list of processed docs
    processed_docs = []
    # vocab contains (term, term id) pairs
    vocab = {}
    # total_tokens stores the total number of tokens
    total_tokens = 0
    for raw_doc in docs:
        # norm_doc stores the normalized tokens of a doc
        norm_doc = []
        if stem == True:
            tokenized_sentence = raw_doc.split(" ")
        else:
            tokenized_sentence = nltk.word_tokenize(raw_doc)##tokenize
        for token in tokenized_sentence:
            if lower == True:
                token = token.lower()
            if stem == True:
                token = stemmer.stem(token.lower())
            if not (token in vocab.keys()):
                vocab[token] = len(vocab) ##add into the vocab,len(vocab) will be the id
            norm_doc.append(token)
            total_tokens += 1
        processed_docs.append(norm_doc)
        
    doc_term_freqs = []
    for doc in processed_docs:
        doc_term_freqs.append(Counter(doc))
        
    invindex = InvertedIndex(vocab, doc_term_freqs)
    
    return invindex

def doc_eval(setname,predict):
    wrong = []
    miss = 0
    total_len = 0
    for key in setname:
        total_len += len(predict[key])
        for evi in setname[key]['evidence']:
            title = unicodedata.normalize('NFC', evi[0])
            if title not in predict[key]:
                if key not in wrong:
                    wrong.append(key)
                    miss += 1

        #for i in wrong:
        #    print("==============Claim id: ", i , " ============")
        #    print("Claim : ", setname[i]['claim'])
        #    print("Target evidence : ", setname[i]['evidence'])
        #    print("Guess document : ", predict[i])
   # return miss,wrong,total_len  
    print("Recall : ", 1 - miss/len(setname))
    print("Average Number : ", total_len/len(setname))

def query_sim(query, index, k , th = 0):
    # scores stores doc ids and their scores
    scores = Counter()
    query = preprocessed_claim_sentence(query)
    query = nltk.word_tokenize(query) 
    for word in query:
        word = stemmer.stem(word.lower())
        if word in index.vocab:  #The word will not be counted if the word doesn't exist in the vocab,
            for i in range(0,len(index.docids(word))):
                doc_id = index.docids(word)[i]
                dts = 1/sqrt(index.doc_len[doc_id]) * log(1 + index.freqs(word)[i]) * log(index.num_docs() / index.f_t(word)) #calculate the tf-idf score
                scores[doc_id] = scores[doc_id] + dts #update the score  
    result = []
    if th == 0:
        return(scores.most_common(k))
    else:
        for i in scores.most_common(k):
            if (i[1] >= th) & (len(result)<3):
                result.append(i)
            else:
                return(result)
    return(result)

def sent_eval(target,guess):
    sen_num = 0
    sen_wrong = []
    for i in guess:
        t = target[i]['evidence']
        for e in t:
            e[0] = unicodedata.normalize('NFC',e[0])
            if not e in guess[i]:
                if not i in sen_wrong:
                    sen_wrong.append(i)
                    sen_num +=1
    total_len = 0
    for i in guess:
        total_len+=len(guess[i])
    print("Sentence Selection Result")
    print("Recall : ",1-sen_num/len(guess))
    print("Average length : ",total_len/len(guess))
    #return 1-sen_num/len(guess),total_len/len(guess)

def sentSearch(query,docs,sent_id,wiki = wiki, k = 20, th = 0,  gs = False):
    #docs = []
    #sent_id = []
    #for doc_title in evidence:
    #    for doc in wiki.wiki[doc_title]:
    #        string = ""
    #        for i in wiki.wiki[doc_title][doc].split(",")[1:]:
    #            string += i + " ,"
    #        docs.append(string[:-1])
    #        sent_id.append([doc_title,doc]) 
    index = processed(docs, lower = True, stem = True)
    result = query_sim(query,index,k,th)
    docs = []
    for i in result:
        docs.append(sent_id[i[0]])
    return(docs)

def ss_grid_search(guess,setname,wiki = wiki):
    filt_results_doc = {}
    for key in setname:
        docs,sent_id = getDoc(guess[key],wiki)
        t = sentSearch(setname[key]['claim'],docs,sent_id,k = 100)
        filt_results_doc[key] = t
    return filt_results_doc

def sent_selection_title(TB_docs,setname,wiki=wiki,topk=20,th=0):
    filt_results_doc = {}
    for key in tqdm(setname):
        docs,sent_id = getDoc(TB_docs[key],wiki)
        t = sentSearch(setname[key]['claim'],docs,sent_id,k=topk)
        filt_results_doc[key] = t
    return filt_results_doc

def sent_selection_cont(cont_docs,titles,setname,wiki=wiki,topk=100,th=0):
    filt_results_doc = {}
    for key in tqdm(cont_docs):
        docs = []
        sent_id = []
        cont_title = title_filter_for_cont(cont_docs[key],titles[key])
        for sent in cont_title:
            string = ""
            for i in wiki.single_sent(sent).split(",")[1:]:
                string += i + " ,"
            docs.append(string[:-1])
            sent_id.append(sent)
        t = sentSearch(setname[key]['claim'],docs,sent_id,k=topk,th=th)
        filt_results_doc[key] = t
    return filt_results_doc
    
def title_filter_for_cont(cont_docs,titles, wiki = wiki):
    sent = []
    for doc_title in cont_docs:
        doc_title = unicodedata.normalize('NFC',doc_title)
        for sent_id in wiki.wiki[doc_title]:
            doc = wiki.wiki[doc_title][sent_id]
            for title in titles:
                if title.lower() in doc.lower():
                    sent.append([doc_title,sent_id])
    return sent

def merged_result(sent_sel1,sent_sel2):
    merged = {}
    for i in sent_sel1:
        merged[i] = []
        for j in sent_sel1[i]:
            merged[i].append(j)
        for j in sent_sel2[i]:
            if j not in sent_sel1[i]:
                merged[i].append(j)
    return merged

def sentent_selection(TB_docs,cont_docs,setname,titles,wiki=wiki,k = 30, th = 0.5):
    TB_title = sent_selection_title(TB_docs,setname)
    cont_title = sent_selection_cont(cont_docs,titles,setname,topk = k, th = th)
    final_title = merged_result(TB_title,cont_title)
    return TB_title

def output_senten_result(result,setname,path):
    output,claim_id = getoutput(result,setname)
    with open(path, 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow("TEST!")
        for i in output:
            tsv_writer.writerow(i)
    return output,claim_id
            
def getoutput(result,setname,wiki=wiki):
    output = []
    claim_id = []
    for i in result:
        for evi in result[i]:
            example = []
            claim = setname[i]['claim']
            claim = preprocessed_claim_sentence(claim)
            claim = unicodedata.normalize('NFC',claim)
            example.append(claim)
            example.append(wiki.single_sent(evi))
            output.append(example)
            claim_id.append([i,evi])
    return output,claim_id

In [6]:
def merged_list(doc_list1,doc_list2,n = 5):
    merged_list = []
    for doc in doc_list1:
        merged_list.append(doc)
    for doc in doc_list2:
        if doc not in merged_list:
            merged_list.append(doc)
    return merged_list

def getDoc(evidence,wiki):
    docs = []
    sent_id = []
    for doc_title in evidence:
        for doc in wiki.wiki[doc_title]:
            string = ""
            for i in wiki.wiki[doc_title][doc].split(",")[1:]:
                string += i + " ,"
            docs.append(string[:-1])
            sent_id.append([doc_title,doc])
    return docs,sent_id

def sentSearch(query,docs,sent_id, k = 20, th = 0,  gs = False):
    #docs = []
    #sent_id = []
    #for doc_title in evidence:
    #    for doc in wiki.wiki[doc_title]:
    #        string = ""
    #        for i in wiki.wiki[doc_title][doc].split(",")[1:]:
    #            string += i + " ,"
    #        docs.append(string[:-1])
    #        sent_id.append([doc_title,doc]) 
    index = processed(docs, lower = True, stem = True)
    result = query_sim(query,index,k,th)
    docs = []
    for i in result:
        docs.append(sent_id[i[0]])
    return(docs)

class Examples:
    def __init__(self, examples,wiki):
        self.examples = {}
        for key in examples:
            self.examples[key] = {}
            self.examples[key]['claim'] = examples[key]['claim']
            self.examples[key]['evidence'] = examples[key]['evidence']
            self.examples[key]['label'] = examples[key]['label']
        for key in tqdm(self.examples):
            claim = self.examples[key]['claim']
            self.examples[key]['case_doc'],self.examples[key]['case_title'] = wiki.search(claim,False)
            self.examples[key]['uncase_doc'],self.examples[key]['uncase_title'] = wiki.search(claim,True)
            if len(self.examples[key]['case_doc']) == 0:
                    self.examples[key]['case_doc'] = self.examples[key]['uncase_doc']
            self.examples[key]['merged_case_uncase_doc'] = merged_list(self.examples[key]['case_doc'],self.examples[key]['uncase_doc'])
            docs,sent_id = getDoc(self.examples[key]['case_doc'],wiki)
            self.examples[key]['case_sent'] = sentSearch(claim,docs,sent_id,k=100)
            docs,sent_id = getDoc(self.examples[key]['uncase_doc'],wiki)
            self.examples[key]['uncase_sent'] = sentSearch(claim,docs,sent_id,k=100)
            self.examples[key]['merged_case_uncase_sent'] = merged_list(self.examples[key]['case_sent'],self.examples[key]['uncase_sent'])
    
    def get_ori(self):
        ori = {}
        for key in self.examples:
            ori[key] = {}
            ori[key]['claim'] = self.examples[key]['claim']
            ori[key]['evidence'] = self.examples[key]['evidence']
            ori[key]['label'] = self.examples[key]['label']
        return ori
    
    def get_uncase_doc(self):
        uncase = {}
        for key in self.examples:
            uncase[key] = self.examples[key]['uncase_doc']
        return uncase
    
    def get_case_doc(self):
        case = {}
        for key in self.examples:
            case[key] = self.examples[key]['case_doc']
        return case
    
    def get_case_sent_list(self,k):
        case = {}
        for key in self.examples:
            kk = min(k,len(self.examples[key]['case_sent']))
            case[key] = self.examples[key]['case_sent'][:kk]
        return case
    
    def get_uncase_sent_list(self,k):
        uncase = {}
        for key in self.examples:
            kk = min(k,len(self.examples[key]['uncase_sent']))
            uncase[key] = self.examples[key]['uncase_sent'][:kk]
        return uncase
    
    def get_case_sent_full(self,k):
        case = {}
        for key in self.examples:
            kk = min(k,len(self.examples[key]['case_sent']))
            case[key] = {}
            case[key]['claim'] = self.examples[key]['claim']
            case[key]['evidence'] = self.examples[key]['case_sent'][:kk]
            case[key]['label'] = self.examples[key]['label']
        return case
    
    def get_case_title(self):
        case_title = {}
        for key in self.examples:
            case_title[key] = self.examples[key]['case_title']
            case_title[key] = list(set(case_title[key]))
        return case_title
    
    def get_uncase_sent_full(self,k):
        uncase = {}
        for key in self.examples:
            kk = min(k,len(self.examples[key]['uncase_sent']))
            uncase[key] = {}
            uncase[key]['claim'] = self.examples[key]['claim']
            uncase[key]['evidence'] = self.examples[key]['uncase_sent'][:kk]
            uncase[key]['label'] = self.examples[key]['label']
        return uncase

In [7]:
dev_examples = Examples(dev,wiki)

100%|██████████| 5001/5001 [05:53<00:00, 17.79it/s]


In [8]:
a= merged_list(dev_examples.examples[key]['case_doc'],dev_examples.examples[key]['uncase_doc'])

NameError: name 'key' is not defined

In [ ]:
import pickle
with open(INTERMEDIATE_DATAPATH+"/pylucene_merged_title_content_devset_dict_100.pkl",'rb') as fp:
    merged_devset_dict = pickle.load(fp)
cont_dev ={}
for key in merged_devset_dict:
    cont_dev[key] = merged_devset_dict[key]['matched'][0]
    
case_title = dev_examples.get_case_title()


In [ ]:
def get_topk(results,k):
    new_results = {}
    for key in results:
        new_results[key] = results[key][:k]
    return new_results

def get_final(examples,cont_result):
    setname = examples.get_ori()
    case_title = examples.get_case_title()
    filter_cont_sent = sent_selection_cont(cont_result,case_title,setname, topk = 10)
    case_sent = examples.get_case_sent_list(20)
    uncase_sent = examples.get_uncase_sent_list(5)
    merged_uncase_case = merged_result(uncase_sent,case_sent)
    final = merged_result(merged_uncase_case,filter_cont_sent)
    return final

In [ ]:
#直接跑这句话
filter_cont_sent = sent_selection_cont(filter_cont,case_title,dev, topk = 100)


In [ ]:
filter_cont['91198']

In [ ]:
sent_eval(dev,merged_uncase_case)

In [ ]:
with open(FINAL_DATAPATH + "cli","w") as f:
    json.dump(cli,f)

In [ ]:
test_function = get_final(dev_examples,filter_cont)

In [ ]:
sent_eval(dev,test_function)

In [ ]:
output_dev,claim_id_dev = output_senten_result(final,dev,FINAL_DATAPATH+ "test.tsv")

In [ ]:
with open(FINAL_DATAPATH + "claim_id_dev.json","w") as f:
    json.dump(claim_id_dev,f)

In [ ]:
def alleval(predicted,actual,gs=False):
    NEI = "NOT ENOUGH INFO"
    correct_label = num_instances = 0
    evidence_prec = num_eprec = 0
    evidence_recall = num_erec = 0
    doc_prec = num_dprec = 0
    doc_rec = num_drec = 0

    for ident, arecord in actual.items():
        precord = predicted[ident]

        alabel = arecord['label'].upper()
        plabel = precord['label'].upper()
        if alabel == plabel:
            correct_label += 1
        num_instances += 1

        if alabel != NEI:
            prec = prec_hits = 0
            rec = rec_hits = 0

            aes = arecord['evidence']
            pes = precord['evidence'][:5]
            for pe in pes:
                if pe in aes:
                    prec += 1
                prec_hits += 1

            for ae in aes:
                if ae in pes:
                    rec += 1
                rec_hits += 1

            ads = set(map(lambda ds: ds[0], aes))
            last_pd = None
            dp = ndp = 0
            for pe in pes:
                if not last_pd or pe[0] != last_pd:
                    if pe[0] in ads:
                        dp += 1
                    ndp += 1
                last_pd = pe[0]

            pds = set(map(lambda ds: ds[0], pes))
            dr = ndr = 0
            for ae in ads:
                if ae in pds:
                    dr += 1
                ndr += 1

            if prec_hits > 0:
                evidence_prec += float(prec) / prec_hits
                num_eprec += 1

            if ndp > 0:
                doc_prec += float(dp) / ndp
                num_dprec += 1

            assert rec_hits > 0
            evidence_recall += float(rec) / rec_hits
            num_erec += 1

            assert ndr > 0
            doc_rec += float(dr) / ndr
            num_drec += 1
    accuracy = correct_label / float(num_instances)
    precision = evidence_prec / float(num_eprec) if num_eprec != 0 else 0
    recall = evidence_recall / float(num_erec) if num_erec != 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    doc_precision = doc_prec / float(num_dprec) if num_dprec != 0 else 0
    doc_recall = doc_rec / float(num_drec) if num_drec != 0 else 0
    doc_f1 = 2 * doc_precision * doc_recall / (doc_precision + doc_recall) if doc_precision + doc_recall > 0 else 0
            
    doc=[doc_precision,doc_recall,doc_f1]
    sent = [precision,recall,f1]
    if gs == True:
        return doc,sent,accuracy 
    else:
        
        print('Label Accuracy', '\t\t%.2f%%' % (100 * accuracy))
        print('Sentence Precision', '\t%.2f%%' % (100 * precision))
        print('Sentence Recall', '\t%.2f%%' % (100 * recall))
        print('Sentence F1', '\t\t%.2f%%' % (100 * f1))
        print('Document Precision', '\t%.2f%%' % (100 * doc_precision))
        print('Document Recall', '\t%.2f%%' % (100 * doc_recall))
        print('Document F1', '\t\t%.2f%%' % (100 * doc_f1))
def read_tsv_result(path):
    result = []
    with open(path,'r') as f:
        csv_reader = csv.reader(f, delimiter='\t')
        for row in csv_reader:
            result.append(row)
    return result
def evi_list2str(evi):
    string = evi[0] + " " + str(evi[1])
    return(unicodedata.normalize('NFC',string))
def get_sent_sel_result(model_output,index, target, th=0.999,k = 5):
    result = {}
    result_for_claim = {}
    for i in target:
        result_for_claim[i] = Counter()
    for i,res in enumerate(model_output):
        ind = index[i][0]
        title = evi_list2str(index[i][1])
        result_for_claim[ind][title] = float(res[1])    
    for i in target:
        result[i] = {}
        result[i]['claim'] = target[i]['claim']
        result[i]['label'] = "SUPPORTS"
        result[i]['evidence'] = []
        titles = result_for_claim[i].most_common(k)
        for inx in titles:
            if len(result[i]['evidence']) < 1 :
                title = [unicodedata.normalize('NFD',inx[0].split(" ")[0]),int(inx[0].split(" ")[1])]
                result[i]['evidence'].append(title)
            else:
                if float(inx[1])>th:
                        #a = unicodedata.normalize('NFD',inx[0])
                        #title = [a.split(" ")[0],int(a.split(" ")[1])]
                    title = [unicodedata.normalize('NFD',inx[0].split(" ")[0]),int(inx[0].split(" ")[1])]
                    result[i]['evidence'].append(title)
        #for inx in titles:
         #   if len(result[i]['evidence'])<1:
          #      if float(inx[1]) > 0.99:
           #         title = [unicodedata.normalize('NFD',inx[0].split(" ")[0]),int(inx[0].split(" ")[1])]
            #        result[i]['evidence'].append(title)
        #for inx in titles:
         #   if len(result[i]['evidence'])<1:
          #      if float(inx[1]) > 0.98:
            #        title = [unicodedata.normalize('NFD',inx[0].split(" ")[0]),int(inx[0].split(" ")[1])]
              #      result[i]['evidence'].append(title)
            
    return result
def grid_search_sent(result,index,setname):
    th = {}
    num = 0.9
    max_f1 = 0
    th = 0
    while num< 1:
        after_model = get_sent_sel_result(result,index,setname,th=num)
        doc,sent,acc=alleval(after_model,setname,gs=True)
        if sent[2]>max_f1:
            max_f1 = sent[2]
            th = num
        num += 0.001
    return th
def getoutput_final(result,setname,wiki=wiki):
    output = []
    claim_id = []
    for key in result:
        example = []
        example.append(unicodedata.normalize('NFC',setname[key]['claim']))
        example.append(wiki.multi_sents(result[key]['evidence']))
        output.append(example)
        claim_id.append(key)
    return output,claim_id
def output_final_test(result,setname,path):
    output,claim_id = getoutput_final(result,setname)
    with open(path, 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow("TEST!")
        for i in output:
            tsv_writer.writerow(i)
    return output,claim_id

In [ ]:
sent_sel_result_list1 = read_tsv_result("/home/junw/workspace/Fact_Checking/output/SentSelection/526"+"/test_results.tsv")
list_shawn = read_tsv_result("/home/junw/workspace/codelab/data/final/output_sentence_prediction_result_test_results.tsv")

In [ ]:
def get_re(sent_sel_result_list,dev,claim_id_dev):
    result = {}
    result_for_claim = {}
    for i in dev:
        result_for_claim[i] = Counter()
    for i,res in enumerate(sent_sel_result_list):
        ind = claim_id_dev[i][0]
        title = evi_list2str(claim_id_dev[i][1])
        result_for_claim[ind][title] = float(res[1])    
    for i in dev:
        result[i] = {}
        result[i]['claim'] = dev[i]['claim']
        result[i]['label'] = "SUPPORTS"
        result[i]['evidence'] = []
        titles = result_for_claim[i].most_common(5)
    number = {}
    new_result = {}
    needmore ={}
    for i in dev:
        new_result[i] = {}
        new_result[i]['evidence'] = []
        new_result[i]['claim'] = dev[i]['claim']
        new_result[i]['label'] = "SUPPORTS"
        number[i] = [[],[],[]]
        titles = result_for_claim[i].most_common(5)
        top1 = result_for_claim[i].most_common(1)[0]
        for title in titles:
            if float(title[1])>=0.999:
                number[i][0].append(title[0])
            if float(title[1])>=0.995:
                number[i][1].append(title[0])
            if float(title[1])>=0.95:
                number[i][2].append(title[0])
        if len(number[i][0])>0:
            for title in number[i][0]:
                title= [unicodedata.normalize('NFD',title.split(" ")[0]),int(title.split(" ")[1])]
                new_result[i]['evidence'].append(title)
        else:
            if len(number[i][1])>1:
                for title in number[i][1]:
                    title= [unicodedata.normalize('NFD',title.split(" ")[0]),int(title.split(" ")[1])]
                    new_result[i]['evidence'].append(title)
            else:
                if len(number[i][2])>2:
                    for title in number[i][2]:
                        title= [unicodedata.normalize('NFD',title.split(" ")[0]),int(title.split(" ")[1])]
                        new_result[i]['evidence'].append(title)
                else:
                    #for title in titles:
                     #   if float(title[1])>=0.9:
                      #      title= [unicodedata.normalize('NFD',title[0].split(" ")[0]),int(title[0].split(" ")[1])]
                       #     new_result[i]['evidence'].append(title)
                    if top1[1] > 0.5:
                        title= [unicodedata.normalize('NFD',top1[0].split(" ")[0]),int(top1[0].split(" ")[1])]
                        new_result[i]['evidence'].append(title)
                    title= [unicodedata.normalize('NFD',top1[0].split(" ")[0]),int(top1[0].split(" ")[1])]
                    needmore[i] = []
                    needmore[i].append(title)
    return new_result

In [ ]:
nr= get_re(sent_sel_result_list1,dev,claim_id_dev)
alleval(nr,dev)

In [ ]:
sec_evi = Examples(new_claim_set,wiki)

# 二级

In [ ]:
def get_new_claim(needmore,setname):
    new_claim = {}
    for i in needmore:
        new_claim[i] = {}
        new_claim[i]['claim'] = setname[i]['claim'] + " " + wiki.single_sent(needmore[i][0])
        new_claim[i]['label'] = setname[i]['label']
        new_claim[i]['evidence'] = setname[i]['evidence']
    return new_claim

def get_2nd_set(result,claim_id,setname):
    new_result,needmore = get_re999(result,setname,claim_id)
    new_claim = get_new_claim(needmore,setname)
    new_example = Examples(new_claim,wiki)
    case = new_example.get_case_sent_list(30)
    uncase = new_example.get_uncase_sent_list(6)
    merged_c_un = merged_result(case,uncase)
    output_dev_2nd,claim_id_dev_2nd = output_senten_result(merged_c_un,new_claim,FINAL_DATAPATH+ "test_2nd.tsv")
    return output_dev_2nd,claim_id_dev_2nd

In [ ]:
new_claim = get_new_claim(needmore,dev)
sec_evi = Examples(new_claim,wiki)
case_2nd = sec_evi.get_case_sent_list(30)
sent_eval(new_claim,case_2nd)
uncase_2nd = sec_evi.get_uncase_sent_list(6)
sent_eval(new_claim,uncase_2nd)
merged_list_set = merged_result(case_2nd,uncase_2nd)
sent_eval(new_claim,merged_list_set)
output_dev_2nd,claim_id_dev_2n2nd = output_senten_result(merged_list_set,new_claim,FINAL_DATAPATH+ "test_2nd.tsv")

In [ ]:
sent_sel_result_list_2nd = read_tsv_result("/home/junw/workspace/codelab/data/final"+"/test_results_2nd.tsv")

In [ ]:
sent_sel_result_list_2nd = read_tsv_result("/home/junw/workspace/codelab/data/final"+"/test_results_2nd.tsv")
sent_sel_result_dev_2nd = get_sent_sel_result(sent_sel_result_list_2nd,claim_id_dev_2n2nd,new_claim,th=0.999)
alleval(sent_sel_result_dev_2nd,new_claim)

In [ ]:
new_result,needmore = get_re999(sent_sel_result_list1,dev,claim_id_dev)
#new_result = get_sent_sel_result(sent_sel_result_list1,claim_id_dev,dev)
alleval(new_result,dev)

In [ ]:
len(output_dev_2nd)

In [ ]:
final_2nd = merged_dict(new_result,sent_sel_result_dev_2nd)
alleval(final_2nd,dev)

In [ ]:
import copy
def merged_dict(dic1,dic2):
    mer = {}
    for i in dic1:
        mer[i] = copy.deepcopy(dic1[i])
    for i in dic2:
        for j in dic2[i]['evidence']:
            if j not in mer[i]['evidence']:
                mer[i]['evidence'].append(j)
    return mer

In [ ]:
def get_re999(sent_sel_result_list,dev,claim_id_dev,th = 0.999):
    result = {}
    result_for_claim = {}
    for i in dev:
        result_for_claim[i] = Counter()
    for i,res in enumerate(sent_sel_result_list):
        ind = claim_id_dev[i][0]
        title = evi_list2str(claim_id_dev[i][1])
        result_for_claim[ind][title] = float(res[1])    
    for i in dev:
        result[i] = {}
        result[i]['claim'] = dev[i]['claim']
        result[i]['label'] = "SUPPORTS"
        result[i]['evidence'] = []
        titles = result_for_claim[i].most_common(5)
    number = {}
    new_result = {}
    needmore ={}
    for i in dev:
        new_result[i] = {}
        new_result[i]['evidence'] = []
        new_result[i]['claim'] = dev[i]['claim']
        new_result[i]['label'] = "SUPPORTS"
        number[i] = [[],[],[]]
        titles = result_for_claim[i].most_common(5)
        top1 = result_for_claim[i].most_common(1)[0]
        for title in titles:
            if float(title[1])>=th:
                number[i][0].append(title[0])
        if len(number[i][0])>0:
            for title in number[i][0]:
                title= [unicodedata.normalize('NFD',title.split(" ")[0]),int(title.split(" ")[1])]
                new_result[i]['evidence'].append(title)
        else:
            title= [unicodedata.normalize('NFD',top1[0].split(" ")[0]),int(top1[0].split(" ")[1])]
            #new_result[i]['evidence'].append(title)
            needmore[i] = []
            needmore[i].append(title)
    return new_result,needmore

In [ ]:
new_result,needmore = get_re999(sent_sel_result_list1,dev,claim_id_dev,th=0.999)
#new_result = get_sent_sel_result(sent_sel_result_list1,claim_id_dev,dev)
alleval(new_result,dev)

In [ ]:
 sent_sel_result_list_2nd = read_tsv_result("/home/junw/workspace/codelab/data/final"+"/test_results_2nd.tsv")
sent_sel_result_dev_2nd,nomore = get_re999(sent_sel_result_list_2nd,new_claim,claim_id_dev_2n2nd,th=0.9996)
alleval(sent_sel_result_dev_2nd,new_claim)

In [ ]:
final_re = merged_dict(new_result,sent_sel_result_dev_2nd)

In [ ]:
alleval(final_re,dev)

In [ ]:
    result = {}
    result_for_claim = {}
    result_for_claim_2nd = {}
    merged_result = {}
    for i in dev:
        result_for_claim[i] = Counter()
    for i in dev:
        result_for_claim_2nd[i] = Counter()
        merged_result[i] = Counter()
    for i,res in enumerate(sent_sel_result_list1):
        ind = claim_id_dev[i][0]
        title = evi_list2str(claim_id_dev[i][1])
        result_for_claim[ind][title] = float(res[1])
        merged_result[ind][title] = float(res[1])
    for i,res in enumerate(sent_sel_result_list_2nd):
        ind = claim_id_dev_2n2nd[i][0]
        title = evi_list2str(claim_id_dev_2n2nd[i][1])
        result_for_claim_2nd[ind][title] = float(res[1])
        merged_result[ind][title] = max(merged_result[ind][title],result_for_claim[ind].most_common(1)[0][1]*float(res[1]))
    #for i in dev:
      #  if result_for_claim_2nd[i].most_common(1) != []:
        #    title  = result_for_claim_2nd[i].most_common(1)[0][0]
          #  sco = result_for_claim_2nd[i].most_common(1)[0][1]
            #merged_result[i][title] = max(merged_result[i][title],result_for_claim[i].most_common(1)[0][1]*sco)
    number = {}
    new_result = {}
    needmore ={}
    for i in dev:
        new_result[i] = {}
        new_result[i]['evidence'] = []
        new_result[i]['claim'] = dev[i]['claim']
        new_result[i]['label'] = "SUPPORTS"
        number[i] = [[],[],[]]
        titles = result_for_claim[i].most_common(5)
        top1 = result_for_claim[i].most_common(1)[0]
        titles_merged = merged_result[i].most_common(5)
        for title in titles_merged:
            if float(title[1])>=0.999:
                number[i][0].append(title[0])
            if float(title[1])>=0.996:
                number[i][1].append(title[0])
            if float(title[1])>=0.99:
                number[i][2].append(title[0])
        if len(number[i][0])>0:
            for title in number[i][0]:
                title= [unicodedata.normalize('NFD',title.split(" ")[0]),int(title.split(" ")[1])]
                new_result[i]['evidence'].append(title)
        else:
            if len(number[i][1])>1:
                for title in number[i][1]:
                    title= [unicodedata.normalize('NFD',title.split(" ")[0]),int(title.split(" ")[1])]
                    new_result[i]['evidence'].append(title)
            else:
                if len(number[i][2])>2:
                    for title in number[i][2]:
                        title= [unicodedata.normalize('NFD',title.split(" ")[0]),int(title.split(" ")[1])]
                        new_result[i]['evidence'].append(title)
                else:
                    if top1[1] > 0.9:
                        title= [unicodedata.normalize('NFD',top1[0].split(" ")[0]),int(top1[0].split(" ")[1])]
                        new_result[i]['evidence'].append(title)
                   #  for title in titles_merged:   
                    #    if float(title[1]) >= 0.9:
                       #     title= [unicodedata.normalize('NFD',top1[0].split(" ")[0]),int(top1[0].split(" ")[1])]
                         #   new_result[i]['evidence'].append(title)

In [ ]:
alleval(new_result,dev)

In [ ]:
nr  = get_sent_sel_result(sent_sel_result_list1,claim_id_dev,dev,th=0.999)
alleval(nr,dev)

# FINAL CLASSIFICATION

In [ ]:
def alleval(predicted,actual,gs=False):
    NEI = "NOT ENOUGH INFO"
    correct_label = num_instances = 0
    evidence_prec = num_eprec = 0
    evidence_recall = num_erec = 0
    doc_prec = num_dprec = 0
    doc_rec = num_drec = 0

    for ident, arecord in actual.items():
        precord = predicted[ident]

        alabel = arecord['label'].upper()
        plabel = precord['label'].upper()
        if alabel == plabel:
            correct_label += 1
        num_instances += 1

        if alabel != NEI:
            prec = prec_hits = 0
            rec = rec_hits = 0

            aes = arecord['evidence']
            pes = precord['evidence'][:5]
            for pe in pes:
                if pe in aes:
                    prec += 1
                prec_hits += 1

            for ae in aes:
                if ae in pes:
                    rec += 1
                rec_hits += 1

            ads = set(map(lambda ds: ds[0], aes))
            last_pd = None
            dp = ndp = 0
            for pe in pes:
                if not last_pd or pe[0] != last_pd:
                    if pe[0] in ads:
                        dp += 1
                    ndp += 1
                last_pd = pe[0]

            pds = set(map(lambda ds: ds[0], pes))
            dr = ndr = 0
            for ae in ads:
                if ae in pds:
                    dr += 1
                ndr += 1

            if prec_hits > 0:
                evidence_prec += float(prec) / prec_hits
                num_eprec += 1

            if ndp > 0:
                doc_prec += float(dp) / ndp
                num_dprec += 1

            assert rec_hits > 0
            evidence_recall += float(rec) / rec_hits
            num_erec += 1

            assert ndr > 0
            doc_rec += float(dr) / ndr
            num_drec += 1
    accuracy = correct_label / float(num_instances)
    precision = evidence_prec / float(num_eprec) if num_eprec != 0 else 0
    recall = evidence_recall / float(num_erec) if num_erec != 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    doc_precision = doc_prec / float(num_dprec) if num_dprec != 0 else 0
    doc_recall = doc_rec / float(num_drec) if num_drec != 0 else 0
    doc_f1 = 2 * doc_precision * doc_recall / (doc_precision + doc_recall) if doc_precision + doc_recall > 0 else 0
            
    doc=[doc_precision,doc_recall,doc_f1]
    sent = [precision,recall,f1]
    if gs == True:
        return doc,sent,accuracy 
    else:
        
        print('Label Accuracy', '\t\t%.2f%%' % (100 * accuracy))
        print('Sentence Precision', '\t%.2f%%' % (100 * precision))
        print('Sentence Recall', '\t%.2f%%' % (100 * recall))
        print('Sentence F1', '\t\t%.2f%%' % (100 * f1))
        print('Document Precision', '\t%.2f%%' % (100 * doc_precision))
        print('Document Recall', '\t%.2f%%' % (100 * doc_recall))
        print('Document F1', '\t\t%.2f%%' % (100 * doc_f1))
def read_tsv_result(path):
    result = []
    with open(path,'r') as f:
        csv_reader = csv.reader(f, delimiter='\t')
        for row in csv_reader:
            result.append(row)
    return result
def evi_list2str(evi):
    string = evi[0] + " " + str(evi[1])
    return(unicodedata.normalize('NFC',string))
def get_sent_sel_result(model_output,index, target, th=0.999,k = 5):
    result = {}
    result_for_claim = {}
    for i in target:
        result_for_claim[i] = Counter()
    for i,res in enumerate(model_output):
        ind = index[i][0]
        title = evi_list2str(index[i][1])
        result_for_claim[ind][title] = float(res[1])    
    for i in target:
        result[i] = {}
        result[i]['claim'] = target[i]['claim']
        result[i]['label'] = "SUPPORTS"
        result[i]['evidence'] = []
        titles = result_for_claim[i].most_common(k)
        for inx in titles:
            if len(result[i]['evidence']) < 1 :
                title = [unicodedata.normalize('NFD',inx[0].split(" ")[0]),int(inx[0].split(" ")[1])]
                result[i]['evidence'].append(title)
            else:
                if float(inx[1])>th:
                    #a = unicodedata.normalize('NFD',inx[0])
                    #title = [a.split(" ")[0],int(a.split(" ")[1])]
                    title = [unicodedata.normalize('NFD',inx[0].split(" ")[0]),int(inx[0].split(" ")[1])]
                    result[i]['evidence'].append(title)
    return result
def grid_search_sent(result,index,setname):
    th = {}
    num = 0.9
    max_f1 = 0
    th = 0
    while num< 1:
        after_model = get_sent_sel_result(result,index,setname,th=num)
        doc,sent,acc=alleval(after_model,setname,gs=True)
        if sent[2]>max_f1:
            max_f1 = sent[2]
            th = num
        num += 0.001
    return th
def getoutput_final(result,setname,wiki=wiki):
    output = []
    claim_id = []
    for key in result:
        example = []
        example.append(preprocessed_claim_sentence(unicodedata.normalize('NFC',setname[key]['claim'])))
        example.append(wiki.multi_sents(result[key]['evidence']))
        output.append(example)
        claim_id.append(key)
    return output,claim_id
def output_final_test(result,setname,path):
    output,claim_id = getoutput_final(result,setname)
    with open(path, 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow("TEST!")
        for i in output:
            tsv_writer.writerow(i)

In [ ]:
def getLabel(res):
    res_new = []
    for i in res:
        res_new.append(float(i))
    num = res_new.index(max(res_new))
    if num==0:
        return "REFUTES"
    if num ==1:
        return "SUPPORTS"
    if num == 2:
        return "NOT ENOUGH INFO"
def get_final_result(sen_sel_result,final_result,index):
    for i,res in enumerate(final_result):
        claim_id = index[i]
        sen_sel_result[claim_id]['label'] = getLabel(res)
    return sen_sel_result

def final_output(result,path):
    new_final = {}
    for i in result:
        new_final[i] = {}
        new_final[i]['claim'] = unicodedata.normalize('NFD', result[i]['claim'])
        new_final[i]['label'] = unicodedata.normalize('NFD', result[i]['label'])
        new_evi = []
        for evi in result[i]['evidence']:
            new_evi.append([unicodedata.normalize('NFD', evi[0]),evi[1]])
        new_final[i]['evidence'] = new_evi
    with open(path,"w") as f:
        json.dump(new_final,f)
    return new_final

In [ ]:
nr= get_re(sent_sel_result_list1,dev,claim_id_dev)
alleval(nr,dev)

In [ ]:
nr= get_re(sent_sel_result_list1,dev,claim_id_dev)
alleval(nr,dev)
output_final, cli = getoutput_final(nr,dev)
output_final_test(nr,dev,FINAL_DATAPATH + "final_test.tsv")

In [ ]:
final_result_tsv = read_tsv_result(FINAL_DATAPATH+"/test_results_final.tsv")
final_result = get_final_result(nr,final_result_tsv,cli)

In [ ]:
with open("")

In [ ]:
final_result = get_final_result(nr,final_result_tsv,cli)
alleval(final_result,dev)

In [ ]:
highrecall = get_sent_sel_result(sent_sel_result_list1,claim_id_dev,dev,th = 0.90)
alleval(highrecall,dev)

In [ ]:
output_final_rc, cli_rc = getoutput_final(highrecall,dev)
output_final_test(highrecall,dev,FINAL_DATAPATH + "final_test_reca.tsv")

In [ ]:
final_result_tsv_high = read_tsv_result(FINAL_DATAPATH+"/result_highrecall.tsv")


In [ ]:
final_result_high = get_final_result(nr,final_result_tsv_high,cli_rc)

In [ ]:
wiki.wiki['Floppy_disk'][0]

In [ ]:
final_output(final_result_high,FINAL_DATAPATH+"/testoutput.json")

In [ ]:
alleval(final_result_high,dev)

In [ ]:
nei = []
for i in final_result_high:
    if (final_result_high[i]['label'] == "REFUTES") & (dev[i]['label'] != "REFUTES"):
        for j in dev[i]['e']
        nei.append(i)

In [ ]:
cli_rc[1]

In [ ]:
refutes = []
for i in final_result_high:
    if (final_result_high[i]['label'] != "REFUTES") & (dev[i]['label'] == "REFUTES"):
        refutes.append(i)

In [ ]:
len(refutes)

In [ ]:
def rightevi(evi1,evi2):
    for i in evi1:
        if i not in evi2:
            return False
    return True

In [ ]:
s=0
sw = 0
r=0
rw = 0
ne = 0
nw = 0
a = [0,0,0]
for i in final_result_high:
    if final_result_high[i]['label'] == "SUPPORTS":
        if dev[i]['label'] != "SUPPORTS":
            if rightevi(dev[i]['evidence'],final_result_high[i]['evidence']):
                a[0]+=1
            sw+=1
        s+=1
    if final_result_high[i]['label'] == "REFUTES":
        if dev[i]['label'] != "REFUTES":
            if rightevi(dev[i]['evidence'],final_result_high[i]['evidence']):
                a[1]+=1
            rw+=1
        r+=1
    if final_result_high[i]['label'] == "NOT ENOUGH INFO":
        if dev[i]['label'] != "NOT ENOUGH INFO":
            if rightevi(dev[i]['evidence'],final_result_high[i]['evidence']):
                a[2]+=1
            nw+=1
        ne+=1

In [ ]:
print(a[0],a[1],a[2])

In [ ]:
testrecall = {}
for i in final_result_high:
    testrecall[i] = final_result_high[i]['evidence']

In [ ]:
sent_eval(dev,testrecall)

In [ ]:
        with open('xx.tsv', 'wt') as fp:
            tsv_writer = csv.writer(fp, delimiter='\t')
            for row in examples:
                tsv_writer.writerow(row)
        

In [ ]:
# [claim, content, 0/1/2]
class ClassificationGenerator:
    def __init__(self, wiki, train_examples):
        self.wiki = wiki
        self.train_examples = train_examples

    def output2file(self, _lists):
#         assert nei_evidences_dataset is not None
        with open('/home/shawn/workspace/research/final_codelab/shawn/input/classification/train.tsv', 'wt') as fp:
            tsv_writer = csv.writer(fp, delimiter='\t')
#             tsv_writer.writerow("TEST!")
            for _list in _lists:
#                 claim = preprocessed_claim_sentence(_dict['claim'])
#                 claim = unicodedata.normalize('NFC', claim)
#                 output_list = [claim, _dict['evidence_sentence']]
                tsv_writer.writerow(_list)
        
    def run(self):
        # [claim, doc, label]
        _lists = list()
        sup_cnt = 0
        ref_cnt = 0
        nei_cnt = 0
        for key in self.train_examples.examples.keys():
            info_dict = self.train_examples.examples[key]
            label = info_dict['label']
            claim = info_dict['claim']
            claim = preprocessed_claim_sentence(claim)
            claim = unicodedata.normalize('NFC', claim)
            # 0 refutes 1 supports 2 nei
            t_label = None
            doc = ""
            if label == 'NOT ENOUGH INFO':
                nei_cnt +=1
                t_label = 2
                length = random.choices(population=[1, 2, 3, 4, 5], weights=[0.4, 0.3, 0.1, 0.1, 0.1], k=1)[0]
                case_sents = info_dict['case_sent'][0:length]
                doc = self.wiki.multi_sents(case_sent)
                _lists.append([claim, doc, str(t_label)])
            elif label == 'SUPPORTS' and sup_cnt<=40000:
                sup_cnt += 1 
                t_label = 1
                evidence_sents = info_dict['evidence'][0:5]
                doc = self.wiki.multi_sents(evidence_sents)
                _lists.append([claim, doc, str(t_label)])
            elif label == 'REFUTES':
                # refutes
                ref_cnt += 1
                t_label = 0
                evidence_sents = info_dict['evidence'][0:5]
                doc = self.wiki.multi_sents(evidence_sents)
                _lists.append([claim, doc, str(t_label)])
#             assert t_label is not None
#             assert doc is not ""
#             _lists.append([claim, doc, str(t_label)])
        self.output2file(_lists)
        print(sup_cnt, ref_cnt, nei_cnt)
        return _lists
classificationGenerator = ClassificationGenerator(wiki, train_examples)
_lists = classificationGenerator.run()
print(len(_lists))

In [ ]:
len(train_output)

In [ ]:
result = 0
for i in train_output:
    if i[2] == '1':
        s+=1

In [ ]:
import random
random.shuffle(train_output)

In [ ]:
new_output = []
s = 0
for i in train_output:
    if i[2] != '1':
        new_output.append(i)
    else:
        if s<=50000:
            new_output.append(i)
            s+=1   

In [ ]:
with open('balance_train.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for i in new_output:
        tsv_writer.writerow(i)

In [ ]:
s = 0
r= 0
nei = 0
for i in new_output:
    if i[2] == '0':
        r+=1
    if i[2] == '1':
        s+=1
    if i[2] == '2':
        nei+=1

In [ ]:
print(s,r,nei)

In [ ]:
s/(s+r+nei)

In [ ]:
r/(s+r+nei)

In [ ]:
nei/(s+r+nei)

In [ ]:
50001/(29775+35639+50001)

In [ ]:
train_output = read_tsv_result("/home/junw/workspace/Fact_Checking/output/Recog/final518/train.tsv")

In [ ]:
"Nikolaj Coster-Waldau worked with the Fox Broadcasting Company." in train_output

In [ ]:
for i in train_output:
    if "Nikolaj Coster-Waldau worked with the Fox Broadcasting Company." in i:
        print(i)

In [ ]:
a = ['Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
  'Fox Broadcasting Company , The Fox Broadcasting Company -LRB- often shortened to Fox and stylized as FOX -RRB- is an American English language commercial broadcast television network that is owned by the Fox Entertainment Group subsidiary of 21st Century Fox . Nikolaj Coster-Waldau , He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot .',
  '1'] 

In [ ]:
b= ['Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'Fox Broadcasting Company , The Fox Broadcasting Company -LRB-   often shortened to Fox and stylized as FOX  -RRB is an American English language commercial broadcast television network that is owned by the Fox Entertainment Group subsidiary of 21st Century Fox . Nikolaj Coster-Waldau , He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB-   2008  -RRB , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot .', '1']

In [ ]:
a[0] == b[0]

In [ ]:
wiki.title_tree['fox']['broadcasting']

In [ ]:
wiki.title_tree['university']['of']['sydney']

In [ ]:
wiki.title_tree['university']['of']['melbourne']

In [ ]:
wiki.title_tree['fox']['<$>']

In [ ]:
wiki.title_tree['fox']['broadcasting']

In [ ]:
dev_examples.examples['40226']

In [ ]:
wiki.dertitles('heavy metal music')

In [ ]:
wiki.dertitles('fox')

In [9]:
wiki.wiki.single_sent([''])

In [ ]:
wiki.title_tree['the']['game']['of']